MIT License

Copyright (c) 2021 Taiki Miyagawa and Akinori F. Ebihara

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# SPRT-TANDEM tutorial
- This notebook is supposed to be opened on Google Colab. For details, see the README of this repo.
- GPU accelerator is required. From the "Runtime" tab on top of the notebook, choose "Change runtime type" to select a GPU.
- First, let's upload the repo onto Colab:

In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from itertools import zip_longest

!git clone https://github.com/Akinori-F-Ebihara/SPRT-TANDEM-tutorial
os.chdir('./SPRT-TANDEM-tutorial')

## Generate the Nosaic-MNIST database
- The script ``make_nmnist.py`` will generate train and test Nosaic-MNIST databases.
- Two files, ``./data-directory/nosaic_mnist_train.tfrecords`` and ``./data-directory/nosaic_mnist_test.tfrecords`` will be generated.

In [ ]:
!python make_nmnist_testonly.py

## Visualize example sequential data
- Choose one random test data to visualize
- The shape of one data is _(duration, width, height, channel) = (20, 28, 28, 1)_

In [ ]:
duration=20 # the length of sequential data

import tensorflow as tf
from datasets.data_processing import read_tfrecords_nosaic_mnist,\
    decode_nosaic_mnist, binarize_labels_nosaic_mnist,\
    normalize_images_nosaic_mnist, reshape_for_featext

# Reed the test tfr
def _parse_image_function(example_proto):
        return tf.io.parse_single_example(example_proto, {
                    'video': tf.io.FixedLenFeature([], tf.string),
                    'label': tf.io.FixedLenFeature([],tf.int64)
                    })

record_file_test='./data-directory/nosaic_mnist_test.tfrecords'
batch_size = 1

raw_image_dataset_test = tf.data.TFRecordDataset(record_file_test)
parsed_image_dataset_test = raw_image_dataset_test.map(_parse_image_function)
parsed_image_dataset_test = parsed_image_dataset_test.batch(
    batch_size, drop_remainder=True) ###
    
for iter_b, feats in enumerate(parsed_image_dataset_test):
    x_batch, y_batch = decode_nosaic_mnist(feats, duration=duration)
    if np.random.rand() > 0.5: break

print('Shape of a data in NMNIST:', x_batch[0].shape)
print('Digit label: ', y_batch.numpy())

plt.figure(figsize = (16,8))
for i in range(duration):
  plt.subplot(4, 5, i+1)
  plt.imshow(np.squeeze(x_batch[0, i]))
  plt.xticks([])
  plt.yticks([])
  plt.title(i)

In [ ]:
!pip install tensorflow-addons
!pip install optuna

In [ ]:
!python train_fe_nmnist.py

## Load the pretrained model

## Estimate log-likelihood ratio (LLR) and plot the LLR trajectory

## Plot the speed-accuracy tradeoff (SAT) curve